In [ ]:
from bs4 import BeautifulSoup
import requests
import csv
import sqlite3

product = input("what do you want to find? ")
product = product.replace(" ", "+")
listing = []
custom_header = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36', 
    'Accept-language': 'en-US,en;q=0.7'
}

In [ ]:
#opening amazon and grabbing all the items
amz = 'https://www.amazon.ca/s?k=' + product
res = requests.get(amz, headers= custom_header)
soup = BeautifulSoup(res.content, "html.parser")
print(amz)

In [ ]:
#filtering functions
def cut(items, src):
    for i in range(len(items)):
        src.remove(items[i])

def remove_out_of_stock(catalogue):
    removal = []
    for i in range(len(catalogue)):
        if type(catalogue[i]["price"]) == str:
            removal.append(catalogue[i])
    cut(removal, catalogue)

def remove_duplicate_items(catalogue):
    items = []
    removal = []
    for i in range(len(catalogue)):
        price = catalogue[i]["price"]
        name = catalogue[i]["name"]
        reviews = catalogue[i]["reviews"]
        rating = catalogue[i]["rating"]
        if {"price": price, "name": name, "rating": rating, "reviews": reviews} in items:
            removal.append(catalogue[i])
        else:
            items.append({"price": price, "name": name, "rating": rating, "reviews": reviews})
    cut(removal, catalogue)

def remove_including(word, catalogue):
    removal = []
    for i in range(len(catalogue)):
        if word.lower() in catalogue[i]["name"].lower():
            removal.append(catalogue[i])
    cut(removal, catalogue)

def remove_excluding(word, catalogue):
    removal = []
    for i in range(len(catalogue)):
        if word.lower() not in catalogue[i]["name"].lower():
            removal.append(catalogue[i])
    cut(removal, catalogue)


In [ ]:
#Grabs info straight from search for amazon
def scrape_page(page):
    items = page.find_all("div", attrs={"data-component-type": "s-search-result"})
    for prod in items:
        link = "https://www.amazon.ca" + prod.find("a", attrs={"class": "a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"}).get("href")
        name = prod.find("span", attrs={"class": "a-size-base-plus a-color-base a-text-normal"}).text.strip()
        if prod.find("span", attrs={"class": "a-icon-alt"}) is None:
            rating = "No rating"
            reviews = "No reviews"
        else:
            rating = prod.find("span", attrs={"class": "a-icon-alt"}).text.strip()
            rating = float(rating[:rating.find(" ")])
            reviews = prod.find("span", attrs={"class": "a-size-base s-underline-text"}).text.strip()
            reviews = int(reviews.replace(",", ""))
        if prod.find("span", attrs={"class": "a-offscreen"}) is None:
            price = "Out of stock"
        else:
            price = (prod.find("span", attrs={"class": "a-offscreen"}).text.strip()[1:])
            price = float(price.replace(",", ""))
        listing.append({"price": price, "rating": rating, "reviews": reviews,"name": name, "link": link})


In [ ]:
#Choose filters
def filters(catalogue) -> str:
    cmd = input("Remove items out of stock?")
    if cmd.lower() in ["yea", "yes", "y"]:
        remove_out_of_stock(catalogue)

    cmd = input("Filter items only with certain word in name, no for no filtering:")
    if cmd.lower() not in ["no", "n"]:
        remove_excluding(cmd, catalogue)

    cmd = input("Filter out items including a certain word, no for no filter:")
    if cmd.lower() not in ["no", "n"]:
        remove_including(cmd, catalogue)
        
    cmd = input("Do you want the data as a CSV, SQL db or both?")
    return cmd.lower()

In [ ]:
#Data exporting functions
def as_csv(catalogue):
    with open("data.csv", 'w', newline='', encoding="utf-8") as csvf:
        fields = ["price", "rating", "reviews", "name", "link"]
        writer = csv.DictWriter(csvf, fieldnames= fields)
        writer.writeheader()
        writer.writerows(catalogue)
        
def as_sql(catalogue):
    conn = sqlite3.connect('products.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS products (
            price REAL,
            rating REAL,
            reviews INTEGER,
            name TEXT,
            link TEXT
        )
    ''')
    conn.commit()

    for item in catalogue:
        cursor.execute('''
            INSERT INTO products (price, rating, reviews, name, link)
            VALUES (:price, :rating, :reviews, :name, :link)
        ''', item)

    conn.commit()

    cursor.execute("SELECT * FROM products")
    rows = cursor.fetchall()
    for row in rows:
        print(row)
    
    conn.close()

In [ ]:
#Iterating through pages, add all items to the list, not removing duplicates or items out of stock
scrape_page(soup)
page_num = "2"
current = soup.find("a", attrs={"aria-label": "Go to next page, page " + page_num})
while current is not None:
    req = requests.get("https://www.amazon.ca" + current.get("href"), headers= custom_header)
    print("https://www.amazon.ca" + current.get("href"))
    sauce = BeautifulSoup(req.content, "html.parser")
    scrape_page(sauce)
    page_num = str(int(page_num) + 1)
    current = sauce.find("a", attrs={"aria-label": "Go to next page, page " + page_num})

In [ ]:
#Exporting the data, asks for what filters they want, as .csv or .db
remove_duplicate_items(listing) #always remove dupes cuz yea
output = filters(listing)
if output == "both":
    as_sql(listing)
    as_csv(listing)
elif output == "csv":
    as_csv(listing)
else:
    as_sql(listing)